# Neural Network Enhancement of ORB/RANSAC Outputs

## Obtaining Training Labels

### Conversion of Net Transformation to Incremental Transformations

In [9]:
import pandas as pd
import numpy as np
from utils import *
import matplotlib.pyplot as plt

# Read the data, which contains net transformations.
PATH_TO_INPUT_FILE = "train_labels_one_chain_only.csv"
PATH_TO_OUTPUT_FILE = "train_labels_incremental_transformations.csv"

df = pd.read_csv(PATH_TO_INPUT_FILE)

# Create a new CSV file.
with open(PATH_TO_OUTPUT_FILE, "w") as f:

    # Write the first row of data in df to the new CSV file.
    df.iloc[[0]].to_csv(f, index=False)

    # Iterate through the rows of the dataframe, and calculate the incremental transformations.
    for i in range(1, 100):
        prev_row = df.iloc[i-1]
        curr_row = df.iloc[i]

        chain_id = curr_row["chain_id"]

        T_start = np.array([[prev_row["x"]], [prev_row["y"]], [prev_row["z"]]])
        T_end = np.array([[curr_row["x"]], [curr_row["y"]], [curr_row["z"]]])

        R_start = np.array([prev_row["qw"], prev_row["qx"], prev_row["qy"], prev_row["qz"]])
        R_end = np.array([curr_row["qw"], curr_row["qx"], curr_row["qy"], curr_row["qz"]])

        T_incremental, R_incremental = decompose_transformations(T_start, R_start, T_end, R_end)

        # Write the incremental transformations to the new CSV file.
        f.write(f"{chain_id},{i},{T_incremental[0][0]},{T_incremental[1][0]},{T_incremental[2][0]},{R_incremental[0]},{R_incremental[1]},{R_incremental[2]},{R_incremental[3]}\n")

### (Sanity Check) Conversion Back to Net Transformations

In [10]:
PATH_TO_SANITY_CHECK_FILE = "train_labels_sanity_check.csv"

df = pd.read_csv(PATH_TO_OUTPUT_FILE)

# Create a new CSV file.
with open(PATH_TO_SANITY_CHECK_FILE, "w") as f:

    # Write the first row of data in df to the new CSV file.
    df.iloc[[0]].to_csv(f, index=False)

    # Iterate through the rows of the dataframe, and calculate the net transformation.
    T_net = np.zeros((3, 1))
    R_net = np.array([1, 0, 0, 0])

    for i in range(1, 100):
        curr_row = df.iloc[i]
        chain_id = curr_row["chain_id"]

        T_incremental = np.array([[curr_row["x"]], [curr_row["y"]], [curr_row["z"]]])
        R_incremental = np.array([curr_row["qw"], curr_row["qx"], curr_row["qy"], curr_row["qz"]])

        T_net, R_net = compose_transformations(T_net, R_net, T_incremental, R_incremental)

        # Write the net transformations to the new CSV file.
        f.write(f"{chain_id},{i},{T_net[0][0]},{T_net[1][0]},{T_net[2][0]},{R_net[0]},{R_net[1]},{R_net[2]},{R_net[3]}\n")

/Users/yeojunjie/Documents/NUS Modules/CS3263/Project/spacecraft-pose-estimation/example_src/utils.py:6: RuntimeWarning: invalid value encountered in sqrt
  q[0] = np.sqrt(1 + matrix[0, 0] + matrix[1, 1] + matrix[2, 2]) / 2
